In [50]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Historical weather').getOrCreate()


In [52]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

###Datasets

In [161]:
weather_df = spark.read.option('inferSchema', 'true').option('header', 'true').csv('sample_data/cleaned_weather_data.csv')
weather_df.printSchema()
weather_df.show(5)

root
 |-- date: timestamp (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- tavg: double (nullable = true)
 |-- tmin: double (nullable = true)
 |-- tmax: double (nullable = true)
 |-- wspd: double (nullable = true)
 |-- pres: double (nullable = true)

+-------------------+--------+-------+---------+---------+----+----+----+----+------+
|               date| country|   city| Latitude|Longitude|tavg|tmin|tmax|wspd|  pres|
+-------------------+--------+-------+---------+---------+----+----+----+----+------+
|2018-07-21 00:00:00|Abkhazia|Sukhumi|43.001525|41.023415|23.4|20.9|25.5| 9.3|1009.6|
|2018-07-22 00:00:00|Abkhazia|Sukhumi|43.001525|41.023415|23.5|21.0|25.7| 9.4|1010.0|
|2018-07-23 00:00:00|Abkhazia|Sukhumi|43.001525|41.023415|23.5|21.1|25.5| 8.2|1007.7|
|2018-07-24 00:00:00|Abkhazia|Sukhumi|43.001525|41.023415|24.3|20.8|27.1| 9.3|1004.4|
|2018-07-25 00:00

In [54]:
country_cont_df= spark.read.option('inferSchema', 'true').option('header', 'true').csv('sample_data/cleaned_country_continent.csv')
country_cont_df.printSchema()
country_cont_df.show(5)

root
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)

+--------------+---------+
|       country|continent|
+--------------+---------+
|   Afghanistan|     Asia|
| �land Islands|   Europe|
|       Albania|   Europe|
|       Algeria|   Africa|
|American Samoa|  Oceania|
+--------------+---------+
only showing top 5 rows



###Tasks
1. Correlation between wind speed and average temperature
2. Standard deviation for average temperature for each country in the month of May each year, and find the country with lowest standard deviation.
3. Max temperature that each country has ever had
4. Top 5 hottest days in all Japan.
5. Difference of each country’s average temperature from global average temperature for 2020. (Window Function)
6. Rolling average of wind speed in Japan for the month of May 2021 (Window Function)
7. Find hottest and coldest day of Japan for each month in 2021
8. Find 3 windiest days of Canada for each month  in 2020
9. Lowest wind speed for each continent (join) and the country, date on which it was recorded.


1. Correlation between wind speed and average temperature

In [55]:
weather_df.stat.corr('tavg','wspd')

-0.07108432573608632

wind speed and avg temp seems to have negative correlation.
Insight: Difference in temperature drives wind speed.

2. Standard deviation for average temperature for each country in the month of May each year, and find the country with lowest standard deviation.

In [173]:
avg_temp_may_df = weather_df\
                    .groupBy(year('date').alias('year'),month('date').alias('month'), 'country')\
                    .agg(min('tavg').alias('avg_temp'))\
                    .filter('month == 5')\
                    .orderBy('country', 'year')
avg_temp_may_df.show()
stddev_avg_temp_may_df = avg_temp_may_df\
                            .groupBy('country')\
                            .agg(stddev('avg_temp').alias('stddev'))\
                            .orderBy(asc('stddev'))\
                            .select('country',round('stddev', 1).alias('stddev_avg_temp_may')).show()

+----+-----+--------------+--------+
|year|month|       country|avg_temp|
+----+-----+--------------+--------+
|2019|    5|      Abkhazia|    12.0|
|2020|    5|      Abkhazia|    11.5|
|2021|    5|      Abkhazia|    11.1|
|2022|    5|      Abkhazia|    11.5|
|2018|    5| Aland Islands|     6.2|
|2019|    5| Aland Islands|     1.4|
|2020|    5| Aland Islands|     3.5|
|2021|    5| Aland Islands|     2.4|
|2022|    5| Aland Islands|     3.7|
|2018|    5|       Albania|    17.2|
|2019|    5|       Albania|    13.3|
|2020|    5|       Albania|    15.2|
|2021|    5|       Albania|    15.7|
|2022|    5|       Albania|    14.7|
|2018|    5|       Algeria|    14.6|
|2019|    5|       Algeria|    16.3|
|2020|    5|       Algeria|    19.0|
|2021|    5|       Algeria|    17.3|
|2022|    5|       Algeria|    15.4|
|2018|    5|American Samoa|    25.3|
+----+-----+--------------+--------+
only showing top 20 rows

+--------------------+-------------------+
|             country|stddev_avg_temp_may|


3. Max temperature that each country has ever had

In [83]:
max_temp_country_everyyeardf = weather_df\
                    .groupBy(year('date').alias('year'), 'country')\
                    .agg(max('tmax').alias('max_temp')).orderBy('country', 'year')
max_temp_country_everyyeardf.show()
max_temp_country_everhaddf = max_temp_country_everyyeardf\
                            .groupBy('country')\
                            .agg(max('max_temp').alias('maxtemp'))\
                            .orderBy('country')\
                            .select('country','maxtemp').show(20)

+----+-------------+--------+
|year|      country|max_temp|
+----+-------------+--------+
|2018|     Abkhazia|    30.0|
|2019|     Abkhazia|    30.0|
|2020|     Abkhazia|    31.8|
|2021|     Abkhazia|    31.0|
|2022|     Abkhazia|    29.7|
|2018|Aland Islands|    30.3|
|2019|Aland Islands|    30.1|
|2020|Aland Islands|    27.6|
|2021|Aland Islands|    28.1|
|2022|Aland Islands|    26.5|
|2018|      Albania|    36.4|
|2019|      Albania|    38.6|
|2020|      Albania|    37.7|
|2021|      Albania|    42.1|
|2022|      Albania|    38.5|
|2018|      Algeria|    36.4|
|2019|      Algeria|    41.7|
|2020|      Algeria|    39.7|
|2021|      Algeria|    43.3|
|2022|      Algeria|    41.3|
+----+-------------+--------+
only showing top 20 rows

+-------------------+-------+
|            country|maxtemp|
+-------------------+-------+
|           Abkhazia|   31.8|
|      Aland Islands|   30.3|
|            Albania|   42.1|
|            Algeria|   43.3|
|     American Samoa|   34.1|
|             

4. Top 5 hottest days in all Japan.

In [157]:
japan_weather_df=weather_df.orderBy(desc('tmax'))
japan_weather_df=japan_weather_df.filter(japan_weather_df.country=='Japan').show(5)

+-------------------+-------+-----+---------+----------+----+----+----+----+------+
|               date|country| city| Latitude| Longitude|tavg|tmin|tmax|wspd|  pres|
+-------------------+-------+-----+---------+----------+----+----+----+----+------+
|2018-07-23 00:00:00|  Japan|Tokyo|35.709026|139.731992|30.8|28.2|38.6|10.0|1005.4|
|2018-08-02 00:00:00|  Japan|Tokyo|35.709026|139.731992|30.9|26.3|36.9| 9.1|1004.6|
|2022-07-01 00:00:00|  Japan|Tokyo|35.709026|139.731992|29.9|25.0|36.6| 9.1|1014.3|
|2021-08-10 00:00:00|  Japan|Tokyo|35.709026|139.731992|30.6|27.2|36.3|17.7| 998.4|
|2020-08-11 00:00:00|  Japan|Tokyo|35.709026|139.731992|31.5|26.6|36.2|14.8|1012.4|
+-------------------+-------+-----+---------+----------+----+----+----+----+------+
only showing top 5 rows



5. Difference of each country’s average temperature from global average temperature for 2020. (Window Function)

In [138]:
avg_temp_df = weather_df\
                .filter(year('date') == 2020)\
                .groupBy('country')\
                .agg(mean('tavg').alias('avg_temp'))\
                .orderBy('country')

avg_temp_df.show()

global_avg_temp = (weather_df.select(mean('tavg')).collect()[0][0])
global_avg_temp

diff_global_avg_temp_df = avg_temp_df\
                            .withColumn('year', lit(2020))\
                            .withColumn('global_avg_temp', lit(global_avg_temp))\
                            .withColumn('difference', (col('avg_temp') - col('global_avg_temp')))

diff_global_avg_temp_df = diff_global_avg_temp_df.select('year', 'country', 'avg_temp', 'global_avg_temp', 'difference')

diff_global_avg_temp_df.show()


+-------------------+--------------------+
|            country|            avg_temp|
+-------------------+--------------------+
|           Abkhazia|  14.867759562841526|
|      Aland Islands|   8.174520547945201|
|            Albania|   16.16931506849313|
|            Algeria|  19.920491803278694|
|     American Samoa|  27.312295081967196|
|             Angola|  25.862295081967194|
|           Anguilla|  27.498907103825157|
|Antigua and Barbuda|  27.105737704918013|
|          Argentina|   17.58907103825134|
|              Aruba|  28.104918032786887|
|            Austria|  13.514520547945196|
|         Azerbaijan|  15.233606557377055|
|            Bahamas|  25.731232876712333|
|            Bahrain|   27.74262295081968|
|         Bangladesh|  26.095081967213133|
|           Barbados|   27.46803278688526|
|            Belarus|   9.581693989071045|
|            Belgium|   12.37780821917809|
|            Bermuda|  22.143287671232873|
|      Bouvet Island|-0.27942238267148034|
+----------

6. Rolling average of wind speed in Japan for the month of May 2021 (Window Function)


In [174]:
window_spec = Window.partitionBy('country').orderBy('date').rowsBetween(Window.unboundedPreceding, Window.currentRow)

japan_may_2021_df = weather_df\
                        .filter(year('date') == 2021)\
                        .filter(month('date') ==5 )\
                        .filter('country == "Japan"')\
                        .withColumn('rolling_avg', mean('wspd').over(window_spec))\
                        .select('date', 'country', 'wspd', round('rolling_avg', 1).alias('rolling_avg'))

japan_may_2021_df.show()


+-------------------+-------+----+-----------+
|               date|country|wspd|rolling_avg|
+-------------------+-------+----+-----------+
|2021-05-01 00:00:00|  Japan|13.9|       13.9|
|2021-05-02 00:00:00|  Japan|14.3|       14.1|
|2021-05-03 00:00:00|  Japan|13.3|       13.8|
|2021-05-04 00:00:00|  Japan| 9.7|       12.8|
|2021-05-05 00:00:00|  Japan|18.7|       14.0|
|2021-05-06 00:00:00|  Japan| 9.9|       13.3|
|2021-05-07 00:00:00|  Japan| 6.1|       12.3|
|2021-05-08 00:00:00|  Japan|10.0|       12.0|
|2021-05-09 00:00:00|  Japan|15.4|       12.4|
|2021-05-10 00:00:00|  Japan|11.6|       12.3|
|2021-05-11 00:00:00|  Japan| 8.6|       12.0|
|2021-05-12 00:00:00|  Japan|10.9|       11.9|
|2021-05-13 00:00:00|  Japan| 7.1|       11.5|
|2021-05-14 00:00:00|  Japan| 9.4|       11.4|
|2021-05-15 00:00:00|  Japan|13.5|       11.5|
|2021-05-16 00:00:00|  Japan|12.8|       11.6|
|2021-05-17 00:00:00|  Japan|22.0|       12.2|
|2021-05-18 00:00:00|  Japan| 8.9|       12.0|
|2021-05-19 0

7. Find hottest and coldest day of Japan for each month in 2021

In [58]:
window_spec1 = Window.partitionBy('country', year('date'), month('date')).orderBy('tmin')
window_spec2 = Window.partitionBy('country', year('date'), month('date')).orderBy(desc('tmax'))

weather_df\
        .filter('country == "Japan"')\
        .filter(year('date') == 2021)\
        .withColumn('min_rank', dense_rank().over(window_spec1))\
        .withColumn('max_rank', dense_rank().over(window_spec2))\
        .withColumn('is_min_or_max', when(col('min_rank') == 1, lit('min')).when(col('max_rank') == 1, lit('max')))\
        .filter((col('min_rank') == 1) | (col('max_rank') == 1))\
        .select(to_date('date').alias('date'), 'country', 'tmin', 'tmax', 'is_min_or_max')\
        .show()


+----------+-------+----+----+-------------+
|      date|country|tmin|tmax|is_min_or_max|
+----------+-------+----+----+-------------+
|2021-01-16|  Japan| 3.9|17.9|          max|
|2021-01-10|  Japan|-2.8| 7.4|          min|
|2021-02-22|  Japan| 7.3|20.8|          max|
|2021-02-18|  Japan|-0.9| 8.6|          min|
|2021-03-31|  Japan|15.4|22.4|          max|
|2021-03-04|  Japan| 1.9|13.5|          min|
|2021-04-22|  Japan|11.1|25.6|          max|
|2021-04-15|  Japan| 6.1|14.7|          min|
|2021-05-25|  Japan|16.7|27.9|          max|
|2021-05-04|  Japan|10.7|23.4|          min|
|2021-06-08|  Japan|18.8|30.6|          max|
|2021-06-01|  Japan|14.8|24.4|          min|
|2021-07-19|  Japan|24.8|33.5|          max|
|2021-07-04|  Japan|19.2|20.5|          min|
|2021-08-10|  Japan|27.2|36.3|          max|
|2021-08-15|  Japan|18.0|19.7|          min|
|2021-09-23|  Japan|23.1|30.6|          max|
|2021-09-28|  Japan|16.7|24.9|          min|
|2021-09-27|  Japan|16.7|22.9|          min|
|2021-10-0

8. Find 3 windiest days of Canada in 2020

In [69]:
window_spec = Window.partitionBy('country', year('date'), month('date')).orderBy(desc('wspd'))

canda_2020_wspd_df=weather_df\
        .filter('country == "Canada"')\
        .filter(year('date') == 2020)\
        .withColumn('max_rank', dense_rank().over(window_spec))\
        .select(to_date('date').alias('date'), 'country', 'wspd')
canda_2020_wspd_df.orderBy(desc('wspd')).show(3)

+----------+-------+----+
|      date|country|wspd|
+----------+-------+----+
|2020-04-22| Canada|28.5|
|2020-11-16| Canada|27.1|
|2020-03-14| Canada|24.0|
+----------+-------+----+
only showing top 3 rows



9. Highest and lowest wind speed for each continent (join) and the country, date on which it was recorded.

In [172]:
join_expr = weather_df['country'] == country_cont_df['country']

joined_df = weather_df.join(F.broadcast(country_cont_df), join_expr)

window_spec1 = Window.partitionBy('continent').orderBy(asc('wspd'))
window_spec2 = Window.partitionBy('continent').orderBy(desc('wspd'))

lowest_windspeed_continent_df = joined_df\
                            .withColumn('dense_rnk', F.dense_rank().over(window_spec1))\
                            .filter('dense_rnk == 1')\
                            .select('continent', weather_df['country'],'date', col('wspd').alias('lowestwindspeed')).distinct()

highest_windspeed_continent_df = joined_df\
                            .withColumn('dense_rnk', F.dense_rank().over(window_spec2))\
                            .filter('dense_rnk == 1')\
                            .select('continent', weather_df['country'],'date', col('wspd').alias('highestwindspeed')).distinct()
lowest_windspeed_continent_df.show()
highest_windspeed_continent_df.show()


+---------+--------------------+-------------------+---------------+
|continent|             country|               date|lowestwindspeed|
+---------+--------------------+-------------------+---------------+
|   Africa|Central African R...|2018-12-04 00:00:00|            0.8|
|   Europe|              Jersey|2020-05-09 00:00:00|            0.0|
|   Europe|              Jersey|2020-05-10 00:00:00|            0.0|
|   Europe|              Jersey|2020-05-11 00:00:00|            0.0|
|   Europe|              Jersey|2020-05-12 00:00:00|            0.0|
|   Europe|              Jersey|2020-05-14 00:00:00|            0.0|
| Americas|Saint Pierre and ...|2021-07-28 00:00:00|            0.0|
|     Asia|          Bangladesh|2020-10-26 00:00:00|            0.2|
|     Asia|          Bangladesh|2021-12-05 00:00:00|            0.2|
|     Asia|          Bangladesh|2021-12-25 00:00:00|            0.2|
|     Asia|          Bangladesh|2021-12-26 00:00:00|            0.2|
|     Asia|            Thailand|20